# Custom GENO

In [ ]:
import os

os.environ["ARCHDEFS"] = f"{os.path.abspath(os.getcwd())}/../CUTEst/ARCHDefs/"
os.environ["SIFDECODE"] = f"{os.path.abspath(os.getcwd())}/../CUTEst/SIFDecode/"
os.environ["MASTSIF"] = f"{os.path.abspath(os.getcwd())}/../CUTEst/sif/"
os.environ["CUTEST"] = f"{os.path.abspath(os.getcwd())}/../CUTEst/CUTEst/"
os.environ["MYARCH"] = "mac64.osx.gfo"
os.environ["PYCUTEST_CACHE"] = (
    f"{os.path.abspath(os.getcwd())}/../pycutest_cache_holder/"
)

import pycutest

problemNames = pycutest.find_problems(constraints="unconstrained")
print(f"There are {len(problemNames)} unconstrained problems")

for problemName in problemNames[:2]:
    problem = pycutest.import_problem(problemName)
    problem.x0
    f, g = problem.obj(problem.x0, gradient=True)

In [ ]:
from timeit import default_timer as timer
import pycutest
import numpy as np

# import cupy as np  # uncomment this for GPU usage

In [ ]:
def solve(
    problem: pycutest.CUTEstProblem,
    np,
    minimize,
    verbose: int = 10,
    line_search_debug_options=None,
):
    start = timer()
    options = {
        "eps_pg": 1e-5,  # Gradient for early stop
        "max_iter": 3000,
        "verbose": verbose,  # Set it to 0 to fully mute it.
        "max_ls": 30,
        "line_search_debug_options": line_search_debug_options,
        "max_sample_count": 30,
    }

    fandg = lambda x: problem.obj(x, gradient=True)
    result = minimize(fandg, problem.x0, options=options, np=np)

    # assemble solution and map back to original problem
    x = result.x
    elapsed = timer() - start
    print("solving took %.3f sec" % elapsed)
    return result, x

In [ ]:
# for i, problemName in enumerate(problemNames):
#     print(f"{problem.name}, {i}/{len(problemNames)-1}")
#     problem = pycutest.import_problem(problemName)
#     problem.obj(problem.x0, gradient=True)

In [ ]:
problems_to_solve = [123]  # At most 288 # To solve by index
# problems_to_solve = None # To solve all
problemNames = pycutest.find_problems(constraints="unconstrained")
problem_names_to_solve = np.array(problemNames)[problems_to_solve]
# problem_names_to_solve = ["DENSCHNA"]  # To solve by name

print(problem_names_to_solve)

In [ ]:
import genosolver

genosolver.check_version("0.1.0")
if problems_to_solve is not None:
    for problemName in problem_names_to_solve:
        print(f"solving {problemName}")
        problem = pycutest.import_problem(problemName)
        print("Starting")
        result, x = solve(problem, np=np, minimize=genosolver.minimize)
        print(problem.report())

In [ ]:
import bayesian_line_search.GPgenosolver as GPgenosolver
from bayesian_line_search.line_search import LineSearchDebugOptions

do_debug = True
debug_options = LineSearchDebugOptions(
    report_termination_reason=True,
    report_return_value=do_debug,
    report_insufficient_acquisition=do_debug,
    report_invalid_f=do_debug,
    report_acquisition_max=do_debug,
    report_area_reduction=do_debug,
    report_kernel_hyperparameter=do_debug,
    gp_verbose=False,
    plot_gp=do_debug,
)

problemNames = pycutest.find_problems(constraints="unconstrained")
if problems_to_solve is not None:
    r = problem_names_to_solve
else:
    r = problemNames
for problemName in r:
    problem = pycutest.import_problem(problemName)
    print(f"solving {problem.name}")
    result, x = solve(
        problem,
        np=np,
        minimize=GPgenosolver.minimize,
        line_search_debug_options=debug_options,
    )
    print(problem.report())